In [1]:
#https://pandas.pydata.org/docs/user_guide/visualization.html

import pandas as pd
import numpy as np 

In [2]:
df1 = pd.read_json(r'C:\Users\AyanC\OneDrive - Universitetet i Agder\Kalman\Input\pmdata\p12\fitbit\steps.json')

print (df1.head())

             dateTime  value
0 2019-11-01 05:32:00     19
1 2019-11-01 05:33:00      0
2 2019-11-01 05:34:00      0
3 2019-11-01 05:35:00      0
4 2019-11-01 05:36:00      8


In [3]:
df1 = df1.groupby(df1['dateTime'].dt.date, as_index = False)['value'].sum()
df1['steps'] = df1['value']
df1.columns

Index(['value', 'steps'], dtype='object')

In [4]:
df1

,value,steps
0,9503,9503
1,9784,9784
2,11409,11409
3,7290,7290
4,7393,7393
...,...,...
123,3723,3723
124,6255,6255
125,0,0
126,0,0


In [5]:
df2 = pd.read_json(r'C:\Users\AyanC\OneDrive - Universitetet i Agder\Kalman\Input\pmdata\p12\fitbit\very_active_minutes.json')
print (df2.head(10))
df2['VPA'] = df2['value']
df2['value']

    dateTime  value
0 2019-11-01     15
1 2019-11-02      0
2 2019-11-03     33
3 2019-11-04     37
4 2019-11-05      0
5 2019-11-06     17
6 2019-11-07     21
7 2019-11-08     21
8 2019-11-09     45
9 2019-11-10      0


0      15
1       0
2      33
3      37
4       0
       ..
147     0
148     0
149     0
150     0
151     0
Name: value, Length: 152, dtype: int64

In [6]:
df3 = pd.read_json(r'C:\Users\AyanC\OneDrive - Universitetet i Agder\Kalman\Input\pmdata\p12\fitbit\moderately_active_minutes.json')
df3['MPA'] = df3['value']
print (df3.head())

    dateTime  value  MPA
0 2019-11-01     14   14
1 2019-11-02      0    0
2 2019-11-03      5    5
3 2019-11-04     34   34
4 2019-11-05      0    0


In [7]:
df4 = pd.read_json(r'C:\Users\AyanC\OneDrive - Universitetet i Agder\Kalman\Input\pmdata\p12\fitbit\lightly_active_minutes.json')
df4['LPA'] = df4['value']
print (df4.head())

ValueError: Expected object or value

In [ ]:
df5 = pd.read_json(r'C:\Users\AyanC\OneDrive - Universitetet i Agder\Kalman\Input\pmdata\p12\fitbit\sedentary_minutes.json')
df5['sedentary'] = df5['value']
print (df5.head())

In [ ]:
df6 = pd.read_json(r'C:\Users\AyanC\OneDrive - Universitetet i Agder\Kalman\Input\pmdata\p12\fitbit\sleep.json')

print (df6.head())

In [ ]:
df6_1 = pd.read_csv(r'C:\Users\AyanC\OneDrive - Universitetet i Agder\Kalman\Input\pmdata\p12\fitbit\sleep_score.csv')
print (df6_1.head())

In [ ]:
df6_1.columns

In [ ]:
columns = ['duration_score', 'deep_sleep_in_minutes', 'resting_heart_rate', 'restlessness']
df6_1 = df6_1[columns]
df6_1.columns
df6_1

In [ ]:
df7 = pd.read_json(r'C:\Users\AyanC\OneDrive - Universitetet i Agder\Kalman\Input\pmdata\p12\fitbit\heart_rate.json')
df7['HR'] = df7['value']
print (df7.head())

In [ ]:
df8 = pd.read_json(r'C:\Users\AyanC\OneDrive - Universitetet i Agder\Kalman\Input\pmdata\p12\fitbit\exercise.json')

print (df8.head())

In [ ]:
df9 = pd.read_json(r'C:\Users\AyanC\OneDrive - Universitetet i Agder\Kalman\Input\pmdata\p12\fitbit\distance.json')

print (df9.head())

In [ ]:
df10 = pd.read_json(r'C:\Users\AyanC\OneDrive - Universitetet i Agder\Kalman\Input\pmdata\p12\fitbit\calories.json')

print (df10.head())

In [ ]:
df = [df2['dateTime'].fillna(0), df1['steps'].fillna(0), df2['VPA'].fillna(0), df3['MPA'], df4['LPA'], df5['sedentary'], df6_1]
df = pd.concat(df, ignore_index= False, axis= 1)
df = df.fillna(0)
df.reset_index(drop=True, inplace=True)
df

In [ ]:
df.to_csv('p12_merged.csv')

In [ ]:
df = pd.read_csv(r'p12_merged.csv', index_col=0)
df.shape

In [ ]:
dc = df[['dateTime', 'steps']].to_dict('split')
dc['data']

In [ ]:
dc_ = {}
for i in dc['data']:
    v = {i[0]: i[1]}
    dc_.update(v)

dc_

In [ ]:
from datetime import datetime

date_ = datetime.today().strftime('%Y-%m-%d')

print(date_)

idx = pd.date_range('2019-11-01', '2020-03-31')

s = pd.Series(dc_)

s.index = pd.DatetimeIndex(s.index)

s = s.reindex(idx, fill_value=np.nan)

print(s.isnull().sum())

In [ ]:
df.style

In [ ]:
df['sleep_duration'] = 1440 - (df['VPA'] + df['MPA'] + df['LPA'] + df['sedentary'])
df

In [ ]:
corrM = df.corr()
corrM

In [ ]:
def calculate_risk_multi(df):
    if (df['steps'] < 5000) and ((df['VPA']*2 + df['MPA'])*7 < 90) and (df['LPA'] >=0): #sedentary
        if(df['sleep_duration']<420): 
            return 0  #inadequate sleep
        else:
            return 1  #sufficient sleep
    else: #active
        if(df['sleep_duration']<420):
            return 2  #inadequate sleep
        else:
            return 3  #sufficient sleep

df['active'] = df.apply(calculate_risk_multi, axis=1)
df.to_csv('p12_merged_multi.csv')
df

In [ ]:
(df[df['active']==0]).shape #sedentary #inadequate sleep

In [ ]:
(df[df['active']==1]).shape #sedentary #sufficient sleep

In [ ]:
(df[df['active']==2]).shape #active #inadequate sleep

In [ ]:
(df[df['active']==3]).shape #active #sufficient sleep

In [ ]:
len(df[df['active']==3])

In [ ]:
df.shape

In [ ]:
df.plot()

In [ ]:
df.plot.area()

In [ ]:
df.plot.area(stacked=False)

In [ ]:
from pandas.plotting import scatter_matrix

scatter_matrix(df, alpha=0.2, figsize=(16, 16), diagonal="kde")

In [ ]:
df.plot(subplots=True, figsize=(15, 18))

In [ ]:
df.plot(subplots=True, layout=(4, 3), figsize=(15, 18), sharex=False)

In [ ]:
from matplotlib import pyplot

fig, ax = pyplot.subplots(1, 1, figsize=(30, 24))
ax.xaxis.tick_top()  # Display x-axis ticks on top.
df.plot(table=False, ax=ax)

In [ ]:
df.plot.bar(colormap="Greens")

In [ ]:
df.plot.bar()

In [ ]:
df.plot.bar(stacked=True);

In [ ]:
df.plot.barh(stacked=True);

In [ ]:
print(df.values)

In [ ]:
df.plot.box();

In [ ]:
df.columns

In [ ]:
columns = ['steps', 'VPA', 'MPA', 'LPA', 'sedentary', 'duration_score',
       'deep_sleep_in_minutes', 'resting_heart_rate', 'restlessness',
       'sleep_duration', 'active']

df = df[columns]

df

In [ ]:
import matplotlib.pyplot as plt

corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

In [ ]:
X, y = df.iloc[:, :-1], df.iloc[:, -1]
X.shape, y.shape

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
X_train = X_train.values.reshape(X_train.shape[0],1,X_train.shape[1])
X_test = X_test.values.reshape(X_test.shape[0],1,X_test.shape[1])

In [ ]:
X_train.astype('float64')
X_test.astype('float64')
y_train.astype('float64')
y_test.astype('float64')

In [ ]:
X_train.shape, X_test.shape

In [ ]:
x_train

In [ ]:
for window in df.rolling(window=3):
    print(window)

In [ ]:
from tsai.all import *

cls = RocketClassifier()
cls.fit(X_train, y_train)

print(cls.score(X_test, y_test))

In [ ]:
print(cls.predict(X_test))

In [ ]:
cls

In [ ]:
cls = MiniRocketClassifier()
cls.fit(X_train, y_train)

print(cls.score(X_test, y_test))

In [ ]:
cls

In [ ]:
print(cls.predict(X_test))

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(cls, X_train, y_train, cv=10, scoring='accuracy')

print (scores)

avg_score = np.mean(scores)
print(avg_score)

In [ ]:
#https://remykarem.github.io/blog/evaluation-metrics.html

import numpy as np

y_test = y_test.reshape(y_test.shape[0],1)

y_pred = cls.predict(X_test)

y_pred = y_pred.reshape(y_pred.shape[0],1)

print(sklearn.metrics.roc_auc_score(y_test,y_pred))
print(sklearn.metrics.confusion_matrix(y_test,y_pred))
print(sklearn.metrics.classification_report(y_test,y_pred))

In [ ]:
y_pred.shape

In [ ]:
y_test.shape

In [ ]:
df.to_csv('test.csv')

In [ ]:
def prepare_data_with_sliding_window_for_multivariate(window_len, df, params_list, target):
	print()
	print("'PRINT input shape:', patient_patient1.shape")
	print('input shape:', df.shape)
	print('window length', window_len)
	display(df)
	print()
	X, y = SlidingWindow(window_len, get_x=params_list, get_y=target)(df)

	#items = itemify(X, y)
	#print(items[100:112])
	
	print(X[100:106])
	print(y[100:106])
	print(X.shape)
	print(y.shape)
	return X, y

In [ ]:
def prepare_data_with_sliding_window_for_univariate_without_target(window_len, df):
	print()
	print("'PRINT input shape:', patient_patient1.shape")
	print('input shape:', df.shape)
	display(df)
	print()
	X, y = SlidingWindow(window_len)(df)

	#items = itemify(X, y)
	#print(items[100:112])
	
	print(X[100:106])
	print(y[100:106])
	print(X.shape)
	print(y.shape)
	return X, y

In [ ]:
def split_data(y, multi=None):
	print('IN SPLIT DATA')
	if multi:
		print('Multi')
		splits = get_splits(y, valid_size=.2, stratify=True, random_state=23, shuffle=False)
	else: 
		print('Uni')
		splits = get_splits(y, shuffle=False)
	return splits

In [ ]:
window_len = 5

X, y = prepare_data_with_sliding_window_for_univariate_without_target(window_len, df['active'])
splits = split_data(y)

In [ ]:
tfms  = [None, [TSRegression()]]
# Standardizes batch of type TSTensor
batch_tfms = TSStandardize()

dls = get_ts_dls(X.astype('float64'), y.astype('float64'), splits=splits, tfms=tfms, batch_tfms=batch_tfms, bs=64)
dls

In [ ]:
dls.one_batch()

In [ ]:
"""
Choose 'InceptionTime' model as the first model for learning.
"""

learn_inception_time_model = ts_learner(dls, InceptionTime, metrics=[mae, rmse, mse], cbs=ShowGraph())

"""
Model itself finds the best 'learning rate'
"""

learn_inception_time_model.lr_find()

In [ ]:
learn_inception_time_model.fit_one_cycle(50, 1e-2)

In [ ]:
learn_inception_time_model.loss_func

In [ ]:
"""
Choose 'LSTM' model as the second model for learning.
"""

learn_LSTM = ts_learner(dls, LSTM, metrics=[mae, rmse], cbs=ShowGraph())
learn_LSTM.lr_find()

In [ ]:
learn_LSTM.fit_one_cycle(50, 9e-2)

In [ ]:

"""
Choose 'GRU' model as the second model for learning.
"""

learn_GRU = ts_learner(dls, GRU, metrics=[mae, rmse], cbs=ShowGraph())
learn_GRU.lr_find()

In [ ]:
learn_GRU = ts_learner(dls, GRU, metrics=[mae, rmse], cbs=ShowGraph())
learn_GRU.fit_one_cycle(50, 5e-2)

In [ ]:
probas, _, preds = learn_inception_time_model.get_X_preds(X[splits[1]])
print(probas)
print(preds)
skm.mean_squared_error(y[splits[1]], preds, squared=False)

In [ ]:
probas, _, preds = learn_LSTM.get_X_preds(X[splits[1]])
print(probas)
print(preds)
skm.mean_squared_error(y[splits[1]], preds, squared=False)

In [ ]:
probas, _, preds = learn_GRU.get_X_preds(X[splits[1]])
skm.mean_squared_error(y[splits[1]], preds, squared=False)